# Interactive minimal working example
In this notebook you are going to run a single simulation, interactively, using the `simulation module` in `dlensalot/ana/config/examples/`, and our goal is to plot the lensing potential estimate.

# Header

In [1]:
import os
from os.path import join as opj
import numpy as np
import healpy as hp

import matplotlib.pyplot as plt

import lenscarf
from lenscarf.run import run

cond4mpi does not exists
could not setup healpix hack. reverting to standard healpy
MSC: using scarf with 2 OMP threads


# Choose analysis

## We will go step by step and do QE lens reconstruction first

In [2]:
## minimal working example
fn = opj(os.path.dirname(os.path.abspath(lenscarf.__file__)),'ana/config/examples/mwe/conf_mwe_fullsky.py')
print(fn)
ana_mwe = run(config=fn, job_id='QE_lensrec', verbose=False).job

/global/u2/s/sebibel/git/lenscarf/lenscarf/ana/config/examples/mwe/conf_mwe_fullsky.py


# Check configuration
Before running lensing reconstruction, let us look into the configuration file

In [3]:
ana_mwe.__dict__.keys()

dict_keys(['dversion', 'tr', 'k', 'version', 'simidxs', 'simidxs_mf', 'Nmf', 'TEMP_suffix', 'TEMP', 'lm_max_len', 'cls_unl', 'cls_len', 'Lmin', 'zbounds', 'zbounds_len', 'pb_ctr', 'pb_extent', 'lmax_ivf', 'sky_coverage', 'spectrum_type', 'lmin_teb', 'OBD', 'tpl', 'tpl_kwargs', 'lmin_tlm', 'lmin_elm', 'lmin_blm', 'nlev_t', 'nlev_p', 'data_type', 'data_field', 'beam', 'lmax_transf', 'nside', 'sims', 'transferfunction', 'transf_tlm', 'transf_elm', 'transf_blm', 'ftl', 'fel', 'fbl', 'ftl_unl', 'fel_unl', 'fbl_unl', 'qe_tasks', 'qe_lm_max_qlm', 'qe_lmax_qlm', 'lmax_qlm', 'mmax_qlm', 'qe_filter_directional', 'ivfs', 'qlms_dd', 'cg_tol', 'chain_model', 'chain_descr', 'cl_analysis', 'btemplate_perturbative_lensremap', 'it_tasks', 'lm_max_unl', 'it_lm_max_qlm', 'lenjob_pbgeometry', 'lenjob_geometry', 'mf_dirname', 'it_filter_directional', 'sims_MAP', 'itmax', 'iterator_typ', 'lensres', 'mfvar', 'soltn_cond', 'stepper_model', 'cpp', 'dlensalot_model', 'libdir_iterators', 'mf', 'plm', 'mf_resp', 

In [4]:
ana_mwe.sims.__dict__.keys()

dict_keys(['pix_lib_phas', 'nlev_t', 'nlev_p', 'sims_cmb_len', 'cl_transf_T', 'cl_transf_P', 'nside'])

## And how does the simulation data look like?

In [5]:
ana_mwe.sims.get_sim_pmap(0)

 [00:02:10] collecting bands > 00%
 [00:02:11] collecting bands > 00%


(array([ 4.12434467, -2.9311794 ,  3.73770884, ..., -4.04839562,
         2.49361223,  0.78644722]),
 array([ 5.01124793, -3.78710219,  3.75455382, ..., -6.37190179,
         9.6573793 , -7.46562312]))

In [ ]:
ana_mwe.run()

 [00:02:08] collecting bands > 00%


In [ ]:
plm = ana_mwe.get_plm(0, subtract_meanfield=False)

In [ ]:
plt.plot(hp.alm2cl(plm))
plt.xscale('log')
plt.yscale('log')

## iterative lensing reconstruction